## Introduction

This assessment requires me to produce a machine learning model that is trained, tested, and evaluated using a set of secondary data.

Several datasets were explored before selecting a suitable one for this project. The dataset chosen is sourced from the UK Land Registry, a government-maintained open data platform that records property transactions across England and Wales. The dataset contains detailed information on residential property sales, including sale price, date of transfer, property type, location, tenure, and whether the property is newly built.
This data is published by GOV.uk and is free to use. https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads

## Importing the dataset

In [1]:
import pandas as pd
import numpy as np


This code imports the two CSV files and concatenates them into a single DataFrame to create one unified dataset.

In [2]:
df1 = pd.read_csv("../data/pp-2023-part1.csv", header=None)
df2 = pd.read_csv("../data/pp-2023-part2.csv", header=None)

df = pd.concat([df1, df2], ignore_index=True)


In [3]:
# Returns the number of rows and columns in the dataset to show its overall size.
df.shape

(856736, 16)

In [4]:
# Displays the first five rows of the dataset to provide an initial overview of the data structure and values.
df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,{0E082196-CE18-5C09-E063-4704A8C0A10E},221000,2023-09-22 00:00,PL6 6JX,T,N,F,3,NaN,PILLAR WALK,NaN,PLYMOUTH,CITY OF PLYMOUTH,CITY OF PLYMOUTH,A,A
1,{0E082196-CE19-5C09-E063-4704A8C0A10E},228000,2023-08-25 00:00,PL7 1SJ,S,N,F,102,NaN,MERAFIELD ROAD,NaN,PLYMOUTH,CITY OF PLYMOUTH,CITY OF PLYMOUTH,A,A
2,{0E082196-CE1A-5C09-E063-4704A8C0A10E},480000,2023-10-26 00:00,TQ6 0AS,F,N,L,1A,NaN,RIVER VIEW,KINGSWEAR,DARTMOUTH,SOUTH HAMS,DEVON,A,A
3,{0E082196-CE1B-5C09-E063-4704A8C0A10E},625000,2023-07-14 00:00,TQ1 2HB,D,N,F,14,NaN,OXLEA CLOSE,NaN,TORQUAY,TORBAY,TORBAY,A,A
4,{0E082196-CE1C-5C09-E063-4704A8C0A10E},174000,2023-08-04 00:00,PL2 1LL,T,N,F,58,NaN,ST AUBYN AVENUE,NaN,PLYMOUTH,CITY OF PLYMOUTH,CITY OF PLYMOUTH,A,A


In [5]:
# Provides a summary of the dataset including column names, data types, and non-null value counts.
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856736 entries, 0 to 856735
Data columns (total 16 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       856736 non-null  object
 1   1       856736 non-null  int64 
 2   2       856736 non-null  object
 3   3       854495 non-null  object
 4   4       856736 non-null  object
 5   5       856736 non-null  object
 6   6       856736 non-null  object
 7   7       856736 non-null  object
 8   8       118592 non-null  object
 9   9       842512 non-null  object
 10  10      322872 non-null  object
 11  11      856736 non-null  object
 12  12      856736 non-null  object
 13  13      856736 non-null  object
 14  14      856736 non-null  object
 15  15      856736 non-null  object
dtypes: int64(1), object(15)
memory usage: 104.6+ MB


In [6]:
df.isnull().sum()


0          0
1          0
2          0
3       2241
4          0
5          0
6          0
7          0
8     738144
9      14224
10    533864
11         0
12         0
13         0
14         0
15         0
dtype: int64

In [22]:
df.columns


Index(['price', 'date_of_transfer', 'property_type', 'new_build', 'tenure',
       'saon', 'street', 'locality', 'town_city', 'district', 'county',
       'log_price'],
      dtype='object')

In [23]:
df = df.drop(columns=["saon", "street", "locality"], errors="ignore")
# Removes address-related columns with high proportions of missing values that are not useful for price prediction.


In [8]:
# Generates descriptive statistics for numerical features, including mean, standard deviation, and quartiles.
df.describe()


,1
count,8.567360e+05
mean,4.055127e+05
std,1.523214e+06
min,1.000000e+00
25%,1.750000e+05
50%,2.750000e+05
75%,4.250000e+05
max,3.930000e+08


In [9]:
# Remove duplicate rows from the dataset
df = df.drop_duplicates()

In [10]:
# Confirm duplicates have been removed
df.duplicated().sum()

np.int64(0)

In [11]:
df.columns


Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], dtype='int64')

In [12]:
# Assigns official UK Land Registry column names to replace unnamed numeric columns.
df.columns = [
    "transaction_id",
    "price",
    "date_of_transfer",
    "postcode",
    "property_type",
    "new_build",
    "tenure",
    "paon",
    "saon",
    "street",
    "locality",
    "town_city",
    "district",
    "county",
    "ppd_category_type",
    "record_status"
]


In [13]:
# Converts price to numeric format and date_of_transfer to datetime to ensure correct data types for analysis.
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["date_of_transfer"] = pd.to_datetime(df["date_of_transfer"], errors="coerce")


In [14]:
df = df.drop(columns=[
    "transaction_id",  # unique identifier
    "postcode",        # too granular, high cardinality
    "paon"             # address-specific
])
# Removes identifiers and highly granular address fields that do not contribute meaningfully to price prediction.

In [15]:
df["log_price"] = np.log1p(df["price"])
# Applies log transformation to the price variable to reduce skewness and improve model performance.

In [16]:
# Keep only standard residential property transactions
df = df[df["ppd_category_type"] == "A"]

# Keep only valid records
df = df[df["record_status"] == "A"]

# Remove administrative metadata columns
df = df.drop(columns=["ppd_category_type", "record_status"])


In [24]:
categorical_cols = [
    "property_type",
    "new_build",
    "tenure",
    "town_city",
    "district",
    "county"
]
categorical_cols = [c for c in categorical_cols if c in df.columns]


In [25]:
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
# Applies one-hot encoding to categorical variables, converting them into numeric format.


In [26]:
# Displays the updated shape of the dataset after encoding categorical features
df_encoded.shape


(705629, 1600)

In [27]:
# Displays the first few rows to confirm successful encoding
df_encoded.head()


,price,date_of_transfer,log_price,property_type_F,property_type_S,property_type_T,new_build_Y,tenure_L,town_city_ABERAERON,town_city_ABERDARE,...,county_WEST SUSSEX,county_WEST YORKSHIRE,county_WESTMORLAND AND FURNESS,county_WILTSHIRE,county_WINDSOR AND MAIDENHEAD,county_WOKINGHAM,county_WORCESTERSHIRE,county_WREKIN,county_WREXHAM,county_YORK
0,221000,2023-09-22,12.305923,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,228000,2023-08-25,12.337105,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,480000,2023-10-26,13.081543,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,625000,2023-07-14,13.345509,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,174000,2023-08-04,12.066816,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [28]:
# Confirms that all remaining columns are numeric and ML-ready
df_encoded.info()


<class 'pandas.core.frame.DataFrame'>
Index: 705629 entries, 0 to 856735
Columns: 1600 entries, price to county_YORK
dtypes: bool(1597), datetime64[ns](1), float64(1), int64(1)
memory usage: 1.1 GB


In [29]:
print(df.columns.tolist())


['price', 'date_of_transfer', 'property_type', 'new_build', 'tenure', 'town_city', 'district', 'county', 'log_price']
